# Introduction

This notebook will implement the workflow with SketchFormer as the image embedding extraction model. It will contain results for both original and augmented datasets.

# Original Dataset

In [1]:
#!/usr/bin/env python3
# pngs_to_continuous_strokes.py
"""
Convert PNG sketch images into continuous stroke sequences expected by SketchFormer.
Each stroke vector has 5 values: [dx, dy, pen_down, pen_up, pen_end/pad].
Sequences longer than `max_seq_len` are adaptively reduced using average pooling
(for continuous deltas) and voting-based aggregation (for binary pen states).
"""

import os
import numpy as np
import cv2
from skimage.morphology import skeletonize
from glob import glob

# -------------------- Image Preprocessing --------------------
def preprocess_image(path, target_size=None):
    """
    Reads an image, binarizes, inverts (so stroke pixels=1), and skeletonizes.
    """
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise RuntimeError(f"Could not read {path}")
    if target_size:
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    # Adaptive threshold using Otsu
    _, th = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if np.mean(th) > 127:  # ensure black strokes on white bg
        th = 255 - th
    sk = skeletonize((th > 0))
    return (sk.astype(np.uint8) * 255)


# -------------------- Contour Extraction --------------------
def image_to_polylines(img, min_length=5, approx_epsilon=2.0):
    """
    Extracts skeleton contours and approximates them as polylines.
    """
    contours, _ = cv2.findContours(img.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    polylines = []
    for cnt in contours:
        if len(cnt) < min_length:
            continue
        cnt = cnt.squeeze()
        if cnt.ndim != 2:
            continue
        approx = cv2.approxPolyDP(cnt.astype(np.int32), approx_epsilon, False)
        approx = approx.squeeze()
        if approx.ndim != 2:
            continue
        polylines.append(approx.astype(np.float32))
    return polylines


# -------------------- Polyline → Sequence --------------------
def polylines_to_continuous_seq(polylines):
    """
    Convert polylines to [dx, dy, pen_down, pen_up, pen_end] sequence.
    """
    seq = []
    for i, poly in enumerate(polylines):
        if poly.shape[0] < 2:
            continue
        deltas = np.diff(poly, axis=0)  # shape (n-1,2)
        for dx, dy in deltas:
            seq.append([float(dx), float(dy), 1.0, 0.0, 0.0])  # pen_down
        # mark stroke end
        if i < len(polylines) - 1:
            seq.append([0.0, 0.0, 0.0, 1.0, 0.0])  # pen_up
    if not seq:
        seq = [[0.0, 0.0, 0.0, 0.0, 1.0]]  # fallback single token
    return np.array(seq, dtype=np.float32)


# -------------------- Sequence Pooling --------------------
def pooled_pen_state(chunk, idx):
    """
    Decide binary pen state (pen_down/up/end) for a pooled segment.
    Uses hybrid logic: majority vote with fallback if any '1' present.
    """
    avg = np.mean(chunk[:, idx])
    if avg >= 0.5:
        return 1.0
    # elif np.any(chunk[:, idx] > 0.5):
    #     return 1.0
    return 0.0


def average_pool_sequence(seq, target_len):
    """
    Average pool a (N,5) sequence to target_len for SketchFormer compatibility.
    - dx, dy averaged normally
    - pen_* flags aggregated via hybrid voting
    """
    n = len(seq)
    if n <= target_len:
        return seq  # no pooling needed

    step = n / target_len
    pooled = []
    for i in range(target_len):
        start = int(i * step)
        end = int((i + 1) * step)
        chunk = seq[start:end]
        if len(chunk) == 0:
            continue
        dx, dy = np.mean(chunk[:, 0]), np.mean(chunk[:, 1])
        pen_down = pooled_pen_state(chunk, 2)
        pen_up = pooled_pen_state(chunk, 3)
        pen_end = pooled_pen_state(chunk, 4)
        pooled.append([dx, dy, pen_down, pen_up, pen_end])

    return np.array(pooled, dtype=np.float32)


# -------------------- Sequence Padding --------------------
def pad_sequence(seq, max_seq_len):
    """
    Pad sequence to fixed length using [0,0,0,0,1].
    """
    pad_token = [0.0, 0.0, 0.0, 0.0, 1.0]
    if len(seq) > max_seq_len:
        seq = seq[:max_seq_len]
    elif len(seq) < max_seq_len:
        pad = np.tile(pad_token, (max_seq_len - len(seq), 1))
        seq = np.vstack([seq, pad])
    return seq


# -------------------- Conversion Pipeline --------------------
def img_path_to_seq(path, target_size=None, max_seq_len=200):
    """
    Complete conversion from PNG → skeleton → polylines → sequence (pooled + padded).
    """
    sk = preprocess_image(path, target_size=target_size)
    polylines = image_to_polylines(sk, min_length=8, approx_epsilon=2.0)
    seq = polylines_to_continuous_seq(polylines)

    # adaptive pooling if too long
    if len(seq) > max_seq_len:
        seq = average_pool_sequence(seq, max_seq_len)

    seq = pad_sequence(seq, max_seq_len)
    return seq


# -------------------- Main Routine --------------------
def main(png_folder, out_npz, target_size=None, max_seq_len=200):
    paths = sorted(glob(os.path.join(png_folder, '*.png')))
    X, filenames = [], []

    for p in paths:
        try:
            seq = img_path_to_seq(p, target_size=target_size, max_seq_len=max_seq_len)
            X.append(seq)
            filenames.append(os.path.basename(p))
            print(f"✅ Converted {os.path.basename(p)} | Length: {np.sum(seq[..., -1] != 1)}")
        except Exception as e:
            print(f"❌ Failed {p}: {e}")

    if not X:
        raise RuntimeError("No sequences produced.")

    X = np.stack(X, axis=0)  # (N, max_seq_len, 5)
    np.savez(out_npz, x=X, filenames=np.array(filenames))
    print(f"\n💾 Saved dataset: {out_npz}")
    print(f"   Shape: {X.shape}")


# -------------------- Driver Code --------------------
# if __name__ == "__main__":
png_folder = "/home/ayushkum/archimera/inputs/input_png"
out_npz = "/home/ayushkum/archimera/sketchformer/sketchformer_dataset/chunk_0.npz"
main(png_folder=png_folder, out_npz=out_npz)



✅ Converted pdf1.png | Length: 200
✅ Converted pdf2.png | Length: 200
✅ Converted pdf3.png | Length: 200
✅ Converted pdf4.png | Length: 200
✅ Converted pdf5.png | Length: 200
✅ Converted pdf6.png | Length: 200
✅ Converted pdf7.png | Length: 200
✅ Converted pdf8.png | Length: 200

💾 Saved dataset: /home/ayushkum/archimera/sketchformer/sketchformer_dataset/chunk_0.npz
   Shape: (8, 200, 5)


In [2]:
#!/usr/bin/env python3
# extract_sketchformer_embeddings.py
"""
Load SketchFormer model from sketchformer-master, restore checkpoint, and extract embeddings.
"""

import os
import sys
import json
import numpy as np
import tensorflow as tf

sys.path.append("./sketchformer_master")

# make sure repo is on path
from sketchformer_master.models.sketchformer import Transformer as SketchTransformer
from sketchformer_master.utils.hparams import HParams

class DummyDataset:
    """Minimal dataset shim required by model __init__"""
    def __init__(self, max_seq_len=200):
        self.hps = {'use_continuous_data': True, 'max_seq_len': max_seq_len}
        self.n_samples = 1
        self.num_classes = 1
        self.n_classes = 1
        self.tokenizer = None

def load_hparams_from_config(config_path=None):
    """
    Create hparams from default then override with config.json if present.
    """
    hps = SketchTransformer.specific_default_hparams()
    if config_path and os.path.exists(config_path):
        import json
        cfg = json.load(open(config_path))
        for k, v in cfg.items():
            # only update keys that exist; HParams supports item assignment
            try:
                hps[k] = v
            except Exception:
                # ignore unknown keys
                pass
    return hps

def build_model_and_restore(weights_dir, max_seq_len=200, config_json=None):
    dataset = DummyDataset(max_seq_len=max_seq_len)
    hps = load_hparams_from_config(config_json)

    
    # instantiate model
    model = SketchTransformer(hps, dataset, out_dir='.', experiment_id='inference')
    # build model graph: call build_model to create layers
    model.build_model()
    # restore checkpoint
    ckpt = tf.train.Checkpoint(model=model)
    latest = tf.train.latest_checkpoint(weights_dir)
    if latest is None:
        raise RuntimeError(f"No checkpoint found in {weights_dir}")
    ckpt.restore(latest).expect_partial()
    print("[INFO] Restored checkpoint:", latest)
    return model

def extract_embeddings_from_npz(npz_path, model, out_embeddings_path="sketchformer_embeddings.npy", out_map_path="id_mapping.json"):
    d = np.load(npz_path, allow_pickle=True)
    X = d['x']  # shape (N, seq_len, 5)
    filenames = d.get('filenames', None)
    if filenames is None:
        # try fallback
        filenames = [f"img_{i}.png" for i in range(X.shape[0])]
    embeddings = []
    for i in range(X.shape[0]):
        seq = X[i]  # (seq_len, 5)
        # model.encode_from_seq expects a 1D or 2D array representing the sequence (it casts to np.array)
        seq = np.expand_dims(seq, axis=0)
        try:
            out = model.encode_from_seq(seq)
            emb = out['embedding']  # should be (1, lowerdim) or (1, seq_len, ...)
            # ensure emb is numpy
            if hasattr(emb, 'numpy'):
                emb_np = emb.numpy()
            else:
                emb_np = np.array(emb)
            # if emb has shape (1, d) or (1, seq_len, d), reduce to (d,)
            if emb_np.ndim == 3:
                # average across sequence axis if needed
                emb_np = emb_np.mean(axis=1)
            emb_np = emb_np.reshape(-1)
            embeddings.append(emb_np.astype('float32'))
            print(f"[{i+1}/{X.shape[0]}] Embedded {filenames[i]} -> {emb_np.shape}")
        except Exception as e:
            print("ERROR embedding", filenames[i], e)
            # append zeros to keep indices aligned
            embeddings.append(np.zeros((model.hps['lowerdim'],), dtype='float32'))

    embeddings = np.stack(embeddings, axis=0)
    np.save(out_embeddings_path, embeddings)
    # save mapping
    id_map = {i: str(filenames[i]) for i in range(len(filenames))}
    with open(out_map_path, "w") as f:
        json.dump(id_map, f, indent=2)
    print("[INFO] Saved embeddings:", out_embeddings_path, "shape:", embeddings.shape)
    print("[INFO] Saved mapping:", out_map_path)
    return embeddings, id_map

# if __name__ == "__main__":
npz_path = "./sketchformer_dataset/chunk_0.npz"
weights_dir = "./sketch-transformer-tf2-cvpr_tform_cont/weights"
config_json = "./sketch-transformer-tf2-cvpr_tform_cont/config.json"  # optional
model = build_model_and_restore(weights_dir, max_seq_len=200, config_json=config_json)

extract_embeddings_from_npz(npz_path, model, out_embeddings_path="./sketchformer_embeddings.npy", out_map_path="./id_mapping_sketchformer.json")


2025-10-30 11:54:06.867977: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-30 11:54:07.191757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-30 11:54:08.556447: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1761825252.276403  108514 gpu_device.cc:2020] Created device /job:localhost/rep

[INFO] Restored checkpoint: ./sketch-transformer-tf2-cvpr_tform_cont/weights/ckpt-12


/home/ayushkum/archimera/env/lib/python3.12/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'encoder_layer_4' (of type EncoderLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/ayushkum/archimera/env/lib/python3.12/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'encoder_layer_5' (of type EncoderLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/ayushkum/archimera/env/lib/python3.12/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'encoder_layer_6' (of type EncoderLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream 

[1/8] Embedded pdf1.png -> (128,)
[2/8] Embedded pdf2.png -> (128,)
[3/8] Embedded pdf3.png -> (128,)
[4/8] Embedded pdf4.png -> (128,)
[5/8] Embedded pdf5.png -> (128,)
[6/8] Embedded pdf6.png -> (128,)
[7/8] Embedded pdf7.png -> (128,)
[8/8] Embedded pdf8.png -> (128,)
[INFO] Saved embeddings: ./sketchformer_embeddings.npy shape: (8, 128)
[INFO] Saved mapping: ./id_mapping_sketchformer.json


(array([[ 1.010989  , -0.41237274,  0.17688197, ...,  0.66918933,
          0.26845735, -0.75079656],
        [ 0.68106216, -0.65177184, -0.5079062 , ..., -0.03892206,
          0.48827237, -0.72250056],
        [ 1.0475414 , -0.7934085 , -0.10133129, ...,  0.4485203 ,
          0.09297235, -1.0622104 ],
        ...,
        [ 1.1594973 , -0.9593415 , -0.20265836, ...,  0.5945516 ,
         -0.02203569, -0.8526877 ],
        [ 0.4188169 , -0.82615554, -0.21041293, ...,  0.2429265 ,
          1.0349033 , -0.3070404 ],
        [ 0.8352951 , -1.0711412 , -0.29345888, ...,  0.4724831 ,
         -0.2534674 , -1.072026  ]], shape=(8, 128), dtype=float32),
 {0: 'pdf1.png',
  1: 'pdf2.png',
  2: 'pdf3.png',
  3: 'pdf4.png',
  4: 'pdf5.png',
  5: 'pdf6.png',
  6: 'pdf7.png',
  7: 'pdf8.png'})

In [3]:
"""
Store SketchFormer embeddings into a FAISS index (cosine similarity).

Steps:
1. Load precomputed SketchFormer embeddings from .npy file.
2. Normalize them for cosine similarity.
3. Create a FAISS IndexFlatIP (Inner Product) index.
4. Add embeddings and save index + mapping for later retrieval.
"""

import os
import json
import numpy as np
import faiss

def build_faiss_index_from_embeddings(
        embeddings_path: str,
        mapping_path: str,
        index_output: str = "sketchformer_index.faiss",
        normalized: bool = True,
):
    """
    Build and save a FAISS index using cosine similarity.

    ---
    Parameters:
        embeddings_path (str): Path to .npy file containing (N, D) embeddings.
        mapping_path (str): Path to JSON or text file mapping IDs to filenames.
        index_output (str): Output FAISS index file path.
        normalized (bool): Whether to normalize embeddings before indexing.
    ---
    Returns:
        (faiss.Index, dict): FAISS index and ID→filename mapping.
    """

    # Load embeddings
    embeddings = np.load(embeddings_path).astype('float32')
    print(f"✅ Loaded embeddings: {embeddings.shape}")

    # Normalize embeddings for cosine similarity
    if normalized:
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        norms[norms == 0] = 1e-10
        embeddings = embeddings / norms
        print("✅ Normalized embeddings for cosine similarity")

    # Load mapping
    if mapping_path.endswith(".json"):
        with open(mapping_path, "r") as f:
            id_mapping = json.load(f)
    else:
        with open(mapping_path, "r") as f:
            filenames = [line.strip() for line in f.readlines()]
        id_mapping = {i: filenames[i] for i in range(len(filenames))}

    # Create FAISS index (Inner Product for cosine similarity)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    print(f"✅ FAISS index created with {index.ntotal} vectors")

    # Save index and mapping
    faiss.write_index(index, index_output)
    print(f"💾 Saved FAISS index at: {index_output}")

    json_output = os.path.splitext(index_output)[0] + "_mapping.json"
    with open(json_output, "w") as f:
        json.dump(id_mapping, f, indent=2)
    print(f"💾 Saved ID mapping at: {json_output}")

    return index, id_mapping


# if __name__ == "__main__":
embeddings_path = "./sketchformer_embeddings.npy"
mapping_path = "./id_mapping_sketchformer.json"   # adjust if you have another mapping
index_output = "./sketchformer_index.faiss"

index, id_mapping = build_faiss_index_from_embeddings(
    embeddings_path=embeddings_path,
    mapping_path=mapping_path,
    index_output=index_output,
    normalized=True
)


✅ Loaded embeddings: (8, 128)
✅ Normalized embeddings for cosine similarity
✅ FAISS index created with 8 vectors
💾 Saved FAISS index at: ./sketchformer_index.faiss
💾 Saved ID mapping at: ./sketchformer_index_mapping.json


In [8]:
import os
import warnings

# Suppress TensorFlow and Keras logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"   # prevents verbose GPU memory messages
os.environ["TF_CPP_MIN_VLOG_LEVEL"] = "3"          # suppress absl GPU device INFO
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import tensorflow as tf
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

import faiss
import numpy as np
import json
import torch
from PIL import Image
from pngs_to_continuous_strokes import img_path_to_seq
from extract_sketchformer_embeddings import build_model_and_restore

def extract_embedding_from_npz(npz: np.ndarray, model):
    try:
        out = model.encode_from_seq(inp_seq=npz)
        # print(type(out))
        emb = out['embedding']
        if hasattr(emb, 'numpy'):
            emb_np = emb.numpy()
        else:
            emb_np = np.array(emb)
        if emb_np.ndim == 3:
            emb_np = emb_np.mean(axis=1)
        emb_np = emb_np.reshape(-1)
        # print(f"Embedding Generated: {emb_np.shape}")
        return emb_np.astype('float32')
    except Exception as e:
        print(f"ERROR Embedding:- {e}")


# if __name__ == "__main__":
    
weights_dir = "./sketch-transformer-tf2-cvpr_tform_cont/weights"
config_json = "./sketch-transformer-tf2-cvpr_tform_cont/config.json"
faiss_index = "./sketchformer_index.faiss"
mapping_path = "./id_mapping_sketchformer.json"
top_k = 5
query_folder = "/home/ayushkum/archimera/query_png"

# Building Model using weights
model = build_model_and_restore(weights_dir=weights_dir, max_seq_len=200, config_json=config_json)
# print("Loaded lowerdim:", model.hps['lowerdim'])

for fort in os.listdir(query_folder):
    if not fort.lower().endswith('.png'):
        continue
    query_path = os.path.join(query_folder, fort)
    # print(query_path)
    # Converting png to continuous stroke form
    seq = img_path_to_seq(path=query_path, target_size=None, max_seq_len=200)
    # print(f"Sequence Shape before reshaping: {seq.shape}")
    # Adding batch dimension to make it compatible with SKetchFormer
    seq = np.expand_dims(seq, axis=0)
    # print(f"Sequence Shape after reshape: {seq.shape}")
    

    # Extracting Embeddings
    embedding = extract_embedding_from_npz(npz=seq, model=model)
    # Normalizing Embedding for similarity matching
    norms = np.linalg.norm(embedding)
    if norms == 0:
        norms = 1e-10
    emb_norm = embedding / norms
    emb_norm = emb_norm.reshape(1, -1)
    # Loading FAISS index and mapping
    index = faiss.read_index(faiss_index)
    with open(mapping_path, "r") as f:
        id_mapping = json.load(f)
    # Search top-k similar images
    # print("Query embedding dim:", emb_norm.shape[1])
    # print("FAISS index dim:", index.d)
    D, I = index.search(emb_norm, top_k)

    # Prepare results
    results = []
    for rank, (idx, dist) in enumerate(zip(I[0], D[0]), start=1):
        fname = id_mapping.get(str(idx)) or id_mapping.get(idx)
        score = dist
        results.append(
            {
                "rank": rank,
                "filename": fname,
                "score": float(score),
            }
        )
    print(f"\n Top similar sketches for {fort}: ")
    for r in results:
        print(f"{r['rank']}. {r['filename']} - score: {round(r['score'] * 100, 2)}%")
    

[INFO] Restored checkpoint: ./sketch-transformer-tf2-cvpr_tform_cont/weights/ckpt-12

 Top similar sketches for pdf3_SIM.png: 
1. pdf7.png - score: 6.26%
2. pdf2.png - score: 2.8%
3. pdf4.png - score: 1.36%
4. pdf8.png - score: 1.32%
5. pdf3.png - score: 0.59%


# Augmented Dataset

In [9]:
#!/usr/bin/env python3
# pngs_to_continuous_strokes.py
"""
Convert PNG sketch images into continuous stroke sequences expected by SketchFormer.
Each stroke vector has 5 values: [dx, dy, pen_down, pen_up, pen_end/pad].
Sequences longer than `max_seq_len` are adaptively reduced using average pooling
(for continuous deltas) and voting-based aggregation (for binary pen states).
"""

import os
import numpy as np
import cv2
from skimage.morphology import skeletonize
from glob import glob

# -------------------- Image Preprocessing --------------------
def preprocess_image(path, target_size=None):
    """
    Reads an image, binarizes, inverts (so stroke pixels=1), and skeletonizes.
    """
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise RuntimeError(f"Could not read {path}")
    if target_size:
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    # Adaptive threshold using Otsu
    _, th = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if np.mean(th) > 127:  # ensure black strokes on white bg
        th = 255 - th
    sk = skeletonize((th > 0))
    return (sk.astype(np.uint8) * 255)


# -------------------- Contour Extraction --------------------
def image_to_polylines(img, min_length=5, approx_epsilon=2.0):
    """
    Extracts skeleton contours and approximates them as polylines.
    """
    contours, _ = cv2.findContours(img.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    polylines = []
    for cnt in contours:
        if len(cnt) < min_length:
            continue
        cnt = cnt.squeeze()
        if cnt.ndim != 2:
            continue
        approx = cv2.approxPolyDP(cnt.astype(np.int32), approx_epsilon, False)
        approx = approx.squeeze()
        if approx.ndim != 2:
            continue
        polylines.append(approx.astype(np.float32))
    return polylines


# -------------------- Polyline → Sequence --------------------
def polylines_to_continuous_seq(polylines):
    """
    Convert polylines to [dx, dy, pen_down, pen_up, pen_end] sequence.
    """
    seq = []
    for i, poly in enumerate(polylines):
        if poly.shape[0] < 2:
            continue
        deltas = np.diff(poly, axis=0)  # shape (n-1,2)
        for dx, dy in deltas:
            seq.append([float(dx), float(dy), 1.0, 0.0, 0.0])  # pen_down
        # mark stroke end
        if i < len(polylines) - 1:
            seq.append([0.0, 0.0, 0.0, 1.0, 0.0])  # pen_up
    if not seq:
        seq = [[0.0, 0.0, 0.0, 0.0, 1.0]]  # fallback single token
    return np.array(seq, dtype=np.float32)


# -------------------- Sequence Pooling --------------------
def pooled_pen_state(chunk, idx):
    """
    Decide binary pen state (pen_down/up/end) for a pooled segment.
    Uses hybrid logic: majority vote with fallback if any '1' present.
    """
    avg = np.mean(chunk[:, idx])
    if avg >= 0.5:
        return 1.0
    # elif np.any(chunk[:, idx] > 0.5):
    #     return 1.0
    return 0.0


def average_pool_sequence(seq, target_len):
    """
    Average pool a (N,5) sequence to target_len for SketchFormer compatibility.
    - dx, dy averaged normally
    - pen_* flags aggregated via hybrid voting
    """
    n = len(seq)
    if n <= target_len:
        return seq  # no pooling needed

    step = n / target_len
    pooled = []
    for i in range(target_len):
        start = int(i * step)
        end = int((i + 1) * step)
        chunk = seq[start:end]
        if len(chunk) == 0:
            continue
        dx, dy = np.mean(chunk[:, 0]), np.mean(chunk[:, 1])
        pen_down = pooled_pen_state(chunk, 2)
        pen_up = pooled_pen_state(chunk, 3)
        pen_end = pooled_pen_state(chunk, 4)
        pooled.append([dx, dy, pen_down, pen_up, pen_end])

    return np.array(pooled, dtype=np.float32)


# -------------------- Sequence Padding --------------------
def pad_sequence(seq, max_seq_len):
    """
    Pad sequence to fixed length using [0,0,0,0,1].
    """
    pad_token = [0.0, 0.0, 0.0, 0.0, 1.0]
    if len(seq) > max_seq_len:
        seq = seq[:max_seq_len]
    elif len(seq) < max_seq_len:
        pad = np.tile(pad_token, (max_seq_len - len(seq), 1))
        seq = np.vstack([seq, pad])
    return seq


# -------------------- Conversion Pipeline --------------------
def img_path_to_seq(path, target_size=None, max_seq_len=200):
    """
    Complete conversion from PNG → skeleton → polylines → sequence (pooled + padded).
    """
    sk = preprocess_image(path, target_size=target_size)
    polylines = image_to_polylines(sk, min_length=8, approx_epsilon=2.0)
    seq = polylines_to_continuous_seq(polylines)

    # adaptive pooling if too long
    if len(seq) > max_seq_len:
        seq = average_pool_sequence(seq, max_seq_len)

    seq = pad_sequence(seq, max_seq_len)
    return seq


# -------------------- Main Routine --------------------
def main(png_folder, out_npz, target_size=None, max_seq_len=200):
    paths = sorted(glob(os.path.join(png_folder, '*.png')))
    X, filenames = [], []

    for p in paths:
        try:
            seq = img_path_to_seq(p, target_size=target_size, max_seq_len=max_seq_len)
            X.append(seq)
            filenames.append(os.path.basename(p))
            print(f"✅ Converted {os.path.basename(p)} | Length: {np.sum(seq[..., -1] != 1)}")
        except Exception as e:
            print(f"❌ Failed {p}: {e}")

    if not X:
        raise RuntimeError("No sequences produced.")

    X = np.stack(X, axis=0)  # (N, max_seq_len, 5)
    np.savez(out_npz, x=X, filenames=np.array(filenames))
    print(f"\n💾 Saved dataset: {out_npz}")
    print(f"   Shape: {X.shape}")


# -------------------- Driver Code --------------------
# if __name__ == "__main__":
png_folder = "/home/ayushkum/archimera/augmented/input_png"
out_npz = "/home/ayushkum/archimera/sketchformer/sketchformer_dataset/chunk_1.npz"
main(png_folder=png_folder, out_npz=out_npz)



✅ Converted pdf1.png | Length: 200
✅ Converted pdf1_aug1.png | Length: 200
✅ Converted pdf1_aug10.png | Length: 200
✅ Converted pdf1_aug2.png | Length: 200
✅ Converted pdf1_aug3.png | Length: 200
✅ Converted pdf1_aug4.png | Length: 200
✅ Converted pdf1_aug5.png | Length: 200
✅ Converted pdf1_aug6.png | Length: 200
✅ Converted pdf1_aug7.png | Length: 200
✅ Converted pdf1_aug8.png | Length: 200
✅ Converted pdf1_aug9.png | Length: 200
✅ Converted pdf2.png | Length: 200
✅ Converted pdf2_aug1.png | Length: 200
✅ Converted pdf2_aug10.png | Length: 200
✅ Converted pdf2_aug2.png | Length: 200
✅ Converted pdf2_aug3.png | Length: 200
✅ Converted pdf2_aug4.png | Length: 200
✅ Converted pdf2_aug5.png | Length: 200
✅ Converted pdf2_aug6.png | Length: 200
✅ Converted pdf2_aug7.png | Length: 200
✅ Converted pdf2_aug8.png | Length: 200
✅ Converted pdf2_aug9.png | Length: 200
✅ Converted pdf3.png | Length: 200
✅ Converted pdf3_aug1.png | Length: 200
✅ Converted pdf3_aug10.png | Length: 200
✅ Converted 

In [10]:
#!/usr/bin/env python3
# extract_sketchformer_embeddings.py
"""
Load SketchFormer model from sketchformer-master, restore checkpoint, and extract embeddings.
"""

import os
import sys
import json
import numpy as np
import tensorflow as tf

sys.path.append("./sketchformer_master")

# make sure repo is on path
from sketchformer_master.models.sketchformer import Transformer as SketchTransformer
from sketchformer_master.utils.hparams import HParams

class DummyDataset:
    """Minimal dataset shim required by model __init__"""
    def __init__(self, max_seq_len=200):
        self.hps = {'use_continuous_data': True, 'max_seq_len': max_seq_len}
        self.n_samples = 1
        self.num_classes = 1
        self.n_classes = 1
        self.tokenizer = None

def load_hparams_from_config(config_path=None):
    """
    Create hparams from default then override with config.json if present.
    """
    hps = SketchTransformer.specific_default_hparams()
    if config_path and os.path.exists(config_path):
        import json
        cfg = json.load(open(config_path))
        for k, v in cfg.items():
            # only update keys that exist; HParams supports item assignment
            try:
                hps[k] = v
            except Exception:
                # ignore unknown keys
                pass
    return hps

def build_model_and_restore(weights_dir, max_seq_len=200, config_json=None):
    dataset = DummyDataset(max_seq_len=max_seq_len)
    hps = load_hparams_from_config(config_json)

    
    # instantiate model
    model = SketchTransformer(hps, dataset, out_dir='.', experiment_id='inference')
    # build model graph: call build_model to create layers
    model.build_model()
    # restore checkpoint
    ckpt = tf.train.Checkpoint(model=model)
    latest = tf.train.latest_checkpoint(weights_dir)
    if latest is None:
        raise RuntimeError(f"No checkpoint found in {weights_dir}")
    ckpt.restore(latest).expect_partial()
    print("[INFO] Restored checkpoint:", latest)
    return model

def extract_embeddings_from_npz(npz_path, model, out_embeddings_path="sketchformer_embeddings.npy", out_map_path="id_mapping.json"):
    d = np.load(npz_path, allow_pickle=True)
    X = d['x']  # shape (N, seq_len, 5)
    filenames = d.get('filenames', None)
    if filenames is None:
        # try fallback
        filenames = [f"img_{i}.png" for i in range(X.shape[0])]
    embeddings = []
    for i in range(X.shape[0]):
        seq = X[i]  # (seq_len, 5)
        # model.encode_from_seq expects a 1D or 2D array representing the sequence (it casts to np.array)
        seq = np.expand_dims(seq, axis=0)
        try:
            out = model.encode_from_seq(seq)
            emb = out['embedding']  # should be (1, lowerdim) or (1, seq_len, ...)
            # ensure emb is numpy
            if hasattr(emb, 'numpy'):
                emb_np = emb.numpy()
            else:
                emb_np = np.array(emb)
            # if emb has shape (1, d) or (1, seq_len, d), reduce to (d,)
            if emb_np.ndim == 3:
                # average across sequence axis if needed
                emb_np = emb_np.mean(axis=1)
            emb_np = emb_np.reshape(-1)
            embeddings.append(emb_np.astype('float32'))
            print(f"[{i+1}/{X.shape[0]}] Embedded {filenames[i]} -> {emb_np.shape}")
        except Exception as e:
            print("ERROR embedding", filenames[i], e)
            # append zeros to keep indices aligned
            embeddings.append(np.zeros((model.hps['lowerdim'],), dtype='float32'))

    embeddings = np.stack(embeddings, axis=0)
    np.save(out_embeddings_path, embeddings)
    # save mapping
    id_map = {i: str(filenames[i]) for i in range(len(filenames))}
    with open(out_map_path, "w") as f:
        json.dump(id_map, f, indent=2)
    print("[INFO] Saved embeddings:", out_embeddings_path, "shape:", embeddings.shape)
    print("[INFO] Saved mapping:", out_map_path)
    return embeddings, id_map

# if __name__ == "__main__":
npz_path = "./sketchformer_dataset/chunk_1.npz"
weights_dir = "./sketch-transformer-tf2-cvpr_tform_cont/weights"
config_json = "./sketch-transformer-tf2-cvpr_tform_cont/config.json"  # optional
model = build_model_and_restore(weights_dir, max_seq_len=200, config_json=config_json)

extract_embeddings_from_npz(npz_path, model, out_embeddings_path="./augmented_sketchformer_embeddings.npy", out_map_path="./augmented_id_mapping_sketchformer.json")


[INFO] Restored checkpoint: ./sketch-transformer-tf2-cvpr_tform_cont/weights/ckpt-12
[1/88] Embedded pdf1.png -> (128,)
[2/88] Embedded pdf1_aug1.png -> (128,)
[3/88] Embedded pdf1_aug10.png -> (128,)
[4/88] Embedded pdf1_aug2.png -> (128,)
[5/88] Embedded pdf1_aug3.png -> (128,)
[6/88] Embedded pdf1_aug4.png -> (128,)
[7/88] Embedded pdf1_aug5.png -> (128,)
[8/88] Embedded pdf1_aug6.png -> (128,)
[9/88] Embedded pdf1_aug7.png -> (128,)
[10/88] Embedded pdf1_aug8.png -> (128,)
[11/88] Embedded pdf1_aug9.png -> (128,)
[12/88] Embedded pdf2.png -> (128,)
[13/88] Embedded pdf2_aug1.png -> (128,)
[14/88] Embedded pdf2_aug10.png -> (128,)
[15/88] Embedded pdf2_aug2.png -> (128,)
[16/88] Embedded pdf2_aug3.png -> (128,)
[17/88] Embedded pdf2_aug4.png -> (128,)
[18/88] Embedded pdf2_aug5.png -> (128,)
[19/88] Embedded pdf2_aug6.png -> (128,)
[20/88] Embedded pdf2_aug7.png -> (128,)
[21/88] Embedded pdf2_aug8.png -> (128,)
[22/88] Embedded pdf2_aug9.png -> (128,)
[23/88] Embedded pdf3.png -> (

(array([[-0.61615586, -0.30742657,  0.60184425, ..., -0.96678734,
          0.33750892, -1.0956132 ],
        [-0.50324064, -0.01265862,  0.7727757 , ..., -1.1639427 ,
          0.74915946, -1.1675646 ],
        [-0.4512274 , -0.27525562,  0.4812553 , ..., -1.0523536 ,
         -0.097603  , -1.0007298 ],
        ...,
        [-0.91827923, -0.5041115 ,  0.3071432 , ..., -1.1805677 ,
          0.0538047 , -0.78230727],
        [-1.2151366 , -0.31619388,  0.76675296, ..., -0.8838505 ,
          0.47338352, -1.0330502 ],
        [-1.1406724 , -0.64906555,  1.3115425 , ..., -0.973471  ,
         -0.11435312, -1.229326  ]], shape=(88, 128), dtype=float32),
 {0: 'pdf1.png',
  1: 'pdf1_aug1.png',
  2: 'pdf1_aug10.png',
  3: 'pdf1_aug2.png',
  4: 'pdf1_aug3.png',
  5: 'pdf1_aug4.png',
  6: 'pdf1_aug5.png',
  7: 'pdf1_aug6.png',
  8: 'pdf1_aug7.png',
  9: 'pdf1_aug8.png',
  10: 'pdf1_aug9.png',
  11: 'pdf2.png',
  12: 'pdf2_aug1.png',
  13: 'pdf2_aug10.png',
  14: 'pdf2_aug2.png',
  15: 'pdf2_au

In [11]:
"""
Store SketchFormer embeddings into a FAISS index (cosine similarity).

Steps:
1. Load precomputed SketchFormer embeddings from .npy file.
2. Normalize them for cosine similarity.
3. Create a FAISS IndexFlatIP (Inner Product) index.
4. Add embeddings and save index + mapping for later retrieval.
"""

import os
import json
import numpy as np
import faiss

def build_faiss_index_from_embeddings(
        embeddings_path: str,
        mapping_path: str,
        index_output: str = "sketchformer_index.faiss",
        normalized: bool = True,
):
    """
    Build and save a FAISS index using cosine similarity.

    ---
    Parameters:
        embeddings_path (str): Path to .npy file containing (N, D) embeddings.
        mapping_path (str): Path to JSON or text file mapping IDs to filenames.
        index_output (str): Output FAISS index file path.
        normalized (bool): Whether to normalize embeddings before indexing.
    ---
    Returns:
        (faiss.Index, dict): FAISS index and ID→filename mapping.
    """

    # Load embeddings
    embeddings = np.load(embeddings_path).astype('float32')
    print(f"✅ Loaded embeddings: {embeddings.shape}")

    # Normalize embeddings for cosine similarity
    if normalized:
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        norms[norms == 0] = 1e-10
        embeddings = embeddings / norms
        print("✅ Normalized embeddings for cosine similarity")

    # Load mapping
    if mapping_path.endswith(".json"):
        with open(mapping_path, "r") as f:
            id_mapping = json.load(f)
    else:
        with open(mapping_path, "r") as f:
            filenames = [line.strip() for line in f.readlines()]
        id_mapping = {i: filenames[i] for i in range(len(filenames))}

    # Create FAISS index (Inner Product for cosine similarity)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    print(f"✅ FAISS index created with {index.ntotal} vectors")

    # Save index and mapping
    faiss.write_index(index, index_output)
    print(f"💾 Saved FAISS index at: {index_output}")

    json_output = os.path.splitext(index_output)[0] + "_mapping.json"
    with open(json_output, "w") as f:
        json.dump(id_mapping, f, indent=2)
    print(f"💾 Saved ID mapping at: {json_output}")

    return index, id_mapping


# if __name__ == "__main__":
embeddings_path = "./augmented_sketchformer_embeddings.npy"
mapping_path = "./augmented_id_mapping_sketchformer.json"   # adjust if you have another mapping
index_output = "./augmented_sketchformer_index.faiss"

index, id_mapping = build_faiss_index_from_embeddings(
    embeddings_path=embeddings_path,
    mapping_path=mapping_path,
    index_output=index_output,
    normalized=True
)


✅ Loaded embeddings: (88, 128)
✅ Normalized embeddings for cosine similarity
✅ FAISS index created with 88 vectors
💾 Saved FAISS index at: ./augmented_sketchformer_index.faiss
💾 Saved ID mapping at: ./augmented_sketchformer_index_mapping.json


In [12]:
import os
import warnings

# Suppress TensorFlow and Keras logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"   # prevents verbose GPU memory messages
os.environ["TF_CPP_MIN_VLOG_LEVEL"] = "3"          # suppress absl GPU device INFO
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import tensorflow as tf
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

import faiss
import numpy as np
import json
import torch
from PIL import Image
from pngs_to_continuous_strokes import img_path_to_seq
from extract_sketchformer_embeddings import build_model_and_restore

def extract_embedding_from_npz(npz: np.ndarray, model):
    try:
        out = model.encode_from_seq(inp_seq=npz)
        # print(type(out))
        emb = out['embedding']
        if hasattr(emb, 'numpy'):
            emb_np = emb.numpy()
        else:
            emb_np = np.array(emb)
        if emb_np.ndim == 3:
            emb_np = emb_np.mean(axis=1)
        emb_np = emb_np.reshape(-1)
        # print(f"Embedding Generated: {emb_np.shape}")
        return emb_np.astype('float32')
    except Exception as e:
        print(f"ERROR Embedding:- {e}")


# if __name__ == "__main__":
    
weights_dir = "./sketch-transformer-tf2-cvpr_tform_cont/weights"
config_json = "./sketch-transformer-tf2-cvpr_tform_cont/config.json"
faiss_index = "./augmented_sketchformer_index.faiss"
mapping_path = "./augmented_id_mapping_sketchformer.json"
top_k = 10
query_folder = "/home/ayushkum/archimera/query_png"

# Building Model using weights
model = build_model_and_restore(weights_dir=weights_dir, max_seq_len=200, config_json=config_json)
# print("Loaded lowerdim:", model.hps['lowerdim'])

for fort in os.listdir(query_folder):
    if not fort.lower().endswith('.png'):
        continue
    query_path = os.path.join(query_folder, fort)
    # print(query_path)
    # Converting png to continuous stroke form
    seq = img_path_to_seq(path=query_path, target_size=None, max_seq_len=200)
    # print(f"Sequence Shape before reshaping: {seq.shape}")
    # Adding batch dimension to make it compatible with SKetchFormer
    seq = np.expand_dims(seq, axis=0)
    # print(f"Sequence Shape after reshape: {seq.shape}")
    

    # Extracting Embeddings
    embedding = extract_embedding_from_npz(npz=seq, model=model)
    # Normalizing Embedding for similarity matching
    norms = np.linalg.norm(embedding)
    if norms == 0:
        norms = 1e-10
    emb_norm = embedding / norms
    emb_norm = emb_norm.reshape(1, -1)
    # Loading FAISS index and mapping
    index = faiss.read_index(faiss_index)
    with open(mapping_path, "r") as f:
        id_mapping = json.load(f)
    # Search top-k similar images
    # print("Query embedding dim:", emb_norm.shape[1])
    # print("FAISS index dim:", index.d)
    D, I = index.search(emb_norm, top_k)

    # Prepare results
    results = []
    for rank, (idx, dist) in enumerate(zip(I[0], D[0]), start=1):
        fname = id_mapping.get(str(idx)) or id_mapping.get(idx)
        score = dist
        results.append(
            {
                "rank": rank,
                "filename": fname,
                "score": float(score),
            }
        )
    print(f"\n Top similar sketches for {fort}: ")
    for r in results:
        print(f"{r['rank']}. {r['filename']} - score: {round(r['score'] * 100, 2)}%")
    

[INFO] Restored checkpoint: ./sketch-transformer-tf2-cvpr_tform_cont/weights/ckpt-12

 Top similar sketches for pdf3_SIM.png: 
1. pdf8_aug10.png - score: 18.62%
2. pdf8_aug4.png - score: 16.46%
3. pdf8_aug1.png - score: 16.27%
4. pdf8.png - score: 16.27%
5. pdf8_aug8.png - score: 16.05%
6. pdf8_aug3.png - score: 15.37%
7. pdf8_aug9.png - score: 13.21%
8. pdf8_aug2.png - score: 12.87%
9. pdf7.png - score: 12.77%
10. pdf1_aug1.png - score: 12.74%
